In [132]:
filename = r'C:\Users\Alan Horst\MachineLearningNg\Python\data\BreastCancerdata.csv'
#filename = r'C:\Users\Alan Horst\MachineLearningNg\Python\data\ex1data2.csv'

In [133]:
import pandas as pd
import numpy as np
import math
import scipy
dataBC = pd.read_csv(filename)
dataBC.head()
#dataBC.plot(kind='scatter',x='radius_mean',y='smoothness_mean')

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [134]:
#Initialize machine learning algorithm of choice
Linear_Regression = 0 #do not use for classification, linear relationships
Logistic_Regression = 1 #only use for classification

In [135]:
#Choose Data to Input
#DataInputx = ['area', 'occupants']
DataInputx = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
dataBC_X = dataBC[DataInputx].copy()
for i in range(0,len(DataInputx)):
    mean = dataBC_X[DataInputx[i]].mean(axis=0)
    std = dataBC_X[DataInputx[i]].std(axis=0)
    def Norm(df):
        return (df[DataInputx[i]]-std)/mean
    dataBC_X[DataInputx[i]] = dataBC_X.apply(Norm, axis=1)
dataBC_X['zeroeth'] = 1
#print(dataBC_X)
if Logistic_Regression == 1:
    BorM = {'M':1, 'B':0}
    dataBC.diagnosis = [BorM[item] for item in dataBC.diagnosis]

dataBC_Y = dataBC[['diagnosis']].copy()
X = dataBC_X.values
y = dataBC_Y.values
n = len(dataBC_X.columns)
m = len(dataBC_Y.index)

In [137]:
if Linear_Regression == 1:
    # Some gradient descent settings
    num_iters = 1500
    alpha = 0.01
    theta = np.zeros((n,1))
    J = 1/(2*m) * sum(np.square((X.dot(theta))-y))
        
    J_history = []
    iter = 0
    while iter < num_iters:
        temp = theta
        theta = temp - (alpha/(m) * sum(((X.dot(theta)-y).T).dot(X))).reshape((-1, 1))
        #np.append(J_history, computeCost(X, y, theta))
        iter += 1
    print('Theta Result:')
    print(theta)
    test = X[3]
    print('Algorithm Test Result:')
    print(np.dot(theta.T,test))

In [138]:
from scipy.optimize import minimize
regularization = 1 #To minimize effect of overfitting, set = 1
if Logistic_Regression == 1:
    iterations = 1500;
    alpha = 0.01;
    theta = np.zeros(n)
    initial_theta = np.zeros((n,1))
    def sigmoid(x):
        return 1/ (1 + np.exp(-x))
    def h(x, theta):
        return np.array(sigmoid(np.dot(x, theta)))
    sigmoid = np.vectorize(sigmoid)
    def costFunction(theta,x,y, lambda_):
        #m = X.shape[0]

        p1 = np.dot(y.T, np.log(h(x, theta)).reshape(m,1))
        p2 = np.dot((np.ones((m,1)) - y).T, np.log( 1 - h(x, theta)).reshape(m,1))
        summ = (p1 + p2)
        J = -summ[0]/m
        if regularization == 1:
            reg_J = lambda_/(2*m)*sum(np.square(theta[1:]))
            J[1:] = J[1:] + reg_J


        return J
    def gradientVect(theta, x, y, lambda_):
        gradient = []

        theta = theta.reshape(n,1)

        beta = h(x, theta) - y
        
        grad = np.dot(X.T, beta) * 1./m

        if regularization == 1:
            reg = theta[1:] * lambda_/m
            grad[1:] = grad[1:] + reg

        return grad.flatten()
        
    def optimizeTheta(x, y, nLabels, lambda_):

        for i in np.arange(0, nLabels):
            theta = np.zeros((n,1))
            res = minimize(costFunction, theta, args=(x, (y == i)*1, lambda_), method='BFGS',
                       jac=gradientVect, options={'maxiter':300})
            test = X[19]
            print(sigmoid(np.dot(res.x,test)))
        return res
optimizeTheta(X,y,1,0.5)

0.840046028346895


      fun: 0.09510303639053273
 hess_inv: array([[ 42.77066123, -28.96605745,  34.41888698,  29.45993103,
         -8.79318995,  20.11799523,  -2.50650173, -32.19961565,
        -46.5115534 ,  15.23062302, -15.11098123, -32.06840572,
         11.58661814,   9.98209023,  -7.96015028, -28.48057848,
          2.00298181,  33.09068213],
       [-28.96605745,  24.9076177 , -24.5380502 , -14.53043744,
         -0.41658021, -20.48369189,  -6.33924057,  25.20624873,
         39.17197524,  -8.06659126,  22.12584397,  19.92300029,
         -1.88243682,  -4.8465935 ,  12.23168365,  20.00168265,
         -5.9953339 , -29.92737875],
       [ 34.41888698, -24.5380502 ,  30.19251278,  24.86207088,
         -5.06323313,  17.65301296,   1.92785242, -23.69999946,
        -39.73722257,  12.99910771, -12.15285806, -25.2097524 ,
          5.14122321,   6.17115122,  -7.68010149, -23.29542498,
          1.76503309,  28.35630948],
       [ 29.45993103, -14.53043744,  24.86207088,  40.39622458,
        -18.740